In [3]:
import tensorflow.keras as K

2023-03-21 21:28:24.021941: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-21 21:28:24.473655: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-21 21:28:24.529099: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-21 21:28:24.529120: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [10]:
"""Task 0: Inception Block"""

def inception_block(A_prev, filters):
    """builds an inception block as described in Going Deeper with
        Convolutions (2014)
    Args:
        A_prev: output from the previous layer
        filters: tuple or list containing F1, F3R, F3,F5R, F5, FPP, respectively:
            F1: number of filters in the 1x1 convolution
            F3R: number of filters in the 1x1 convolution before the 3x3 convolution
            F3: number of filters in the 3x3 convolution
            F5R: number of filters in the 1x1 convolution before the 5x5 convolution
            F5: number of filters in the 5x5 convolution
            FPP: number of filters in the 1x1 convolution after the max pooling
    Returns:
        concatenated output of the inception block"""
    
    f1, f3r, f3, f5r, f5, fpp = filters

    conv1 = K.layers.Conv2D(filters=f1, kernel_size=(1, 1), padding='same',
                            activation='relu')(A_prev)
    conv2a = K.layers.Conv2D(filters=f3r, kernel_size=(1, 1), padding='same',
                            activation='relu')(A_prev)
    conv2b = K.layers.Conv2D(filters=f3, kernel_size=(3, 3), padding='same',
                            activation='relu')(conv2a)
    conv3a = K.layers.Conv2D(filters=f5r, kernel_size=(1, 1), padding='same',
                             activation='relu')(A_prev)
    conv3b = K.layers.Conv2D(filters=f5, kernel_size=(5, 5), padding='same',
                             activation='relu')(conv3a)
    pool1 = K.layers.MaxPooling2D(pool_size=(3, 3), strides=(1, 1),
                                  padding='same')(A_prev)
    conv4 = K.layers.Conv2D(filters=fpp, kernel_size=(1, 1), padding='same',
                            activation='relu')(pool1)
    output = K.layers.Concatenate()([conv1, conv2b, conv3b, conv4])
    return output

In [11]:
if __name__ == '__main__':
    X = K.Input(shape=(224, 224, 3))
    Y = inception_block(X, [64, 96, 128, 16, 32, 32])
    model = K.models.Model(inputs=X, outputs=Y)
    model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_19 (Conv2D)             (None, 224, 224, 96  384         ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_21 (Conv2D)             (None, 224, 224, 16  64          ['input_4[0][0]']                
                                )                                                             

In [23]:
"""Task 1: Inception Network"""


def inception_network():
    """builds the inception network as described in Going Deeper with
        Convolutions (2014)
    Returns:
        Keras model"""

    init = K.initializers.he_normal()
    inputs = K.Input(shape=(224, 224, 3))
    conv1 = K.layers.Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2),
                            padding='same', activation='relu',
                            kernel_initializer=init)(inputs)
    poo1 = K.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2),
                                 padding='same')(conv1)
    conv2a = K.layers.Conv2D(filters=64, kernel_size=(1, 1), padding='same',
                            activation='relu',
                            kernel_initializer=init)(poo1)
    conv2b = K.layers.Conv2D(filters=192, kernel_size=(3, 3), padding='same',
                            activation='relu',
                            kernel_initializer=init)(conv2a)
    pool2 = K.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2),
                                  padding='same')(conv2b)
    block1 = inception_block(pool2, [64, 96, 128, 16, 32, 32])
    block2 = inception_block(block1, [128, 128, 192, 32, 96, 64])
    pool3 = K.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2),
                                  padding='same')(block2)
    block3 = inception_block(pool3, [192, 96, 208, 16, 48, 64])
    block4 = inception_block(block3, [160, 112, 224, 24, 64, 64])
    block5 = inception_block(block4, [128, 128, 256, 24, 64, 64])
    block6 = inception_block(block5, [112, 144, 288, 32, 64, 64])
    block7 = inception_block(block6, [256, 160, 320, 32, 128, 128])
    pool4 = K.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2),
                                  padding='same')(block7)
    block8 = inception_block(pool4, [256, 160, 320, 32, 128, 128])
    block9 = inception_block(block8, [384, 192, 384, 48, 128, 128])
    pool5 = K.layers.AveragePooling2D(pool_size=(7, 7), strides=(7, 7),
                                      padding='valid')(block9)
    dropout = K.layers.Dropout(rate=0.4)(pool5)
    dense = K.layers.Dense(units=1000, activation='softmax',
                           kernel_initializer=init)(dropout)
    model = K.Model(inputs=inputs, outputs=dense)
    return model

In [24]:
if __name__ == '__main__':
    model = inception_network()
    model.summary()

/home/ediddev/.local/lib/python3.8/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_27 (Conv2D)             (None, 112, 112, 64  9472        ['input_9[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d_4 (MaxPooling2D)  (None, 56, 56, 64)  0           ['conv2d_27[0][0]']              
                                                                                            

In [4]:
"""Task 2: Identity Block"""

def identity_block(A_prev, filters):
    """Builds an identity block as described in Deep Residual Learning for
        Image Recognition (2015)
    Args:
        A_prev: output from the previous layer
        filters: tuple or list containing F11, F3, F12, respectively:
            F11: number of filters in the first 1x1 convolution
            F3: number of filters in the 3x3 convolution
            F12: number of filters in the second 1x1 convolution
    Returns:
        activated output of the identity block"""

    F11, F3, F12 = filters
    init = K.initializers.he_normal()
    conv1 = K.layers.Conv2D(filters=F11, kernel_size=(1, 1), padding='same',
                            kernel_initializer=init)(A_prev)
    bn1 = K.layers.BatchNormalization(axis=3)(conv1)
    act1 = K.layers.Activation('relu')(bn1)
    conv2 = K.layers.Conv2D(filters=F3, kernel_size=(3, 3), padding='same',
                            kernel_initializer=init)(act1)
    bn2 = K.layers.BatchNormalization(axis=3)(conv2)
    act2 = K.layers.Activation('relu')(bn2)
    conv3 = K.layers.Conv2D(filters=F12, kernel_size=(1, 1), padding='same',
                            kernel_initializer=init)(act2)
    bn3 = K.layers.BatchNormalization(axis=3)(conv3)
    add = K.layers.Add()([bn3, A_prev])
    act3 = K.layers.Activation('relu')(add)
    return act3

In [5]:
if __name__ == '__main__':
    X = K.Input(shape=(224, 224, 256))
    Y = identity_block(X, [64, 64, 256])
    model = K.models.Model(inputs=X, outputs=Y)
    model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 2  0           []                               
                                56)]                                                              
                                                                                                  
 conv2d (Conv2D)                (None, 224, 224, 64  16448       ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 224, 224, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                             

2023-03-21 21:28:35.236457: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-21 21:28:35.236549: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-21 21:28:35.236613: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (BOBO-CODE): /proc/driver/nvidia/version does not exist
2023-03-21 21:28:35.238136: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/ediddev/.local/lib/python3.8/site-packages/keras

 batch_normalization_2 (BatchNo  (None, 224, 224, 25  1024       ['conv2d_2[0][0]']               
 rmalization)                   6)                                                                
                                                                                                  
 add (Add)                      (None, 224, 224, 25  0           ['batch_normalization_2[0][0]',  
                                6)                                'input_1[0][0]']                
                                                                                                  
 activation_2 (Activation)      (None, 224, 224, 25  0           ['add[0][0]']                    
                                6)                                                                
                                                                                                  
Total params: 71,552
Trainable params: 70,784
Non-trainable params: 768
_____________________________________

In [6]:
"""Task 3: Projection Block"""

def projection_block(A_prev, filters, s=2):
    """Builds a projection block as described in Deep Residual Learning for
        Image Recognition (2015)
    Args:
        A_prev: output from the previous layer
        filters: tuple or list containing F11, F3, F12, respectively:
            F11: number of filters in the first 1x1 convolution
            F3: number of filters in the 3x3 convolution
            F12: number of filters in the second 1x1 convolution and the 1x1
                convolution in the shortcut connection
        s: stride of the first convolution in both the main path and the
            shortcut connection
    Returns:
        activated output of the projection block"""

    F11, F3, F12 = filters
    init = K.initializers.he_normal()
    conv1 = K.layers.Conv2D(filters=F11, kernel_size=(1, 1), strides=s,
                            padding='same', kernel_initializer=init)(A_prev)
    bn1 = K.layers.BatchNormalization(axis=3)(conv1)
    act1 = K.layers.Activation('relu')(bn1)
    conv2 = K.layers.Conv2D(filters=F3, kernel_size=(3, 3), padding='same',
                            kernel_initializer=init)(act1)
    bn2 = K.layers.BatchNormalization(axis=3)(conv2)
    act2 = K.layers.Activation('relu')(bn2)
    conv3 = K.layers.Conv2D(filters=F12, kernel_size=(1, 1), padding='same',
                            kernel_initializer=init)(act2)
    bn3 = K.layers.BatchNormalization(axis=3)(conv3)
    conv4 = K.layers.Conv2D(filters=F12, kernel_size=(1, 1), strides=s,
                            padding='same', kernel_initializer=init)(A_prev)
    bn4 = K.layers.BatchNormalization(axis=3)(conv4)
    add = K.layers.Add()([bn3, bn4])
    act3 = K.layers.Activation('relu')(add)
    return act3

In [7]:
if __name__ == '__main__':
    X = K.Input(shape=(224, 224, 3))
    Y = projection_block(X, [64, 64, 256])
    model = K.models.Model(inputs=X, outputs=Y)
    model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_3 (Conv2D)              (None, 112, 112, 64  256         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_3 (BatchNo  (None, 112, 112, 64  256        ['conv2d_3[0][0]']               
 rmalization)                   )                                                           

/home/ediddev/.local/lib/python3.8/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [10]:
"""Task 4: ResNet-50"""

def resnet50():
    """Builds the ResNet-50 architecture as described in Deep Residual
        Learning for Image Recognition (2015)
    Returns:
        a Keras model"""

    init = K.initializers.he_normal()
    inputs = K.Input(shape=(224, 224, 3))
    conv1 = K.layers.Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2),
                            padding='same', kernel_initializer=init)(inputs)
    bn1 = K.layers.BatchNormalization(axis=3)(conv1)
    act1 = K.layers.Activation('relu')(bn1)
    pool1 = K.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2),
                                  padding='same')(act1)
    block1 = projection_block(pool1, [64, 64, 256], s=1)
    block2 = identity_block(block1, [64, 64, 256])
    block3 = identity_block(block2, [64, 64, 256])
    
    block4 = projection_block(block3, [128, 128, 512])
    block5 = identity_block(block4, [128, 128, 512])
    block6 = identity_block(block5, [128, 128, 512])
    block7 = identity_block(block6, [128, 128, 512])
    
    block8 = projection_block(block7, [256, 256, 1024])
    block9 = identity_block(block8, [256, 256, 1024])
    block10 = identity_block(block9, [256, 256, 1024])
    block11 = identity_block(block10, [256, 256, 1024])
    block12 = identity_block(block11, [256, 256, 1024])
    block13 = identity_block(block12, [256, 256, 1024])
    
    block14 = projection_block(block13, [512, 512, 2048])
    block15 = identity_block(block14, [512, 512, 2048])
    block16 = identity_block(block15, [512, 512, 2048])
    pool2 = K.layers.AveragePooling2D(pool_size=(7, 7), strides=(7, 7),
                                      padding='valid')(block16)
    dense = K.layers.Dense(units=1000, activation='softmax',
                           kernel_initializer=init)(pool2)
    return K.models.Model(inputs=inputs, outputs=dense)


In [9]:
if __name__ == '__main__':
    model = resnet50()
    model.summary()

/home/ediddev/.local/lib/python3.8/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_7 (Conv2D)              (None, 112, 112, 64  9472        ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_7 (BatchNo  (None, 112, 112, 64  256        ['conv2d_7[0][0]']               
 rmalization)                   )                                                           

In [13]:
"""Task 5: Dense Block"""

def dense_block(X, nb_filters, growth_rate, layers):
    """Builds a dense block as described in Densely Connected Convolutional
        Networks
    Args:
        X: output from the previous layer
        nb_filters: integer representing the number of filters in X
        growth_rate: growth rate for the dense block
        layers: number of layers in the dense block
    Returns:
        The concatenated output of each layer within the Dense Block and the
            number of filters within the concatenated outputs, respectively"""

    init = K.initializers.he_normal()
    for _ in range(layers):
        bn1 = K.layers.BatchNormalization(axis=3)(X)
        act1 = K.layers.Activation('relu')(bn1)
        # bottleneck layer to reduce the number of input freature maps
        conv1 = K.layers.Conv2D(filters=4 * growth_rate, kernel_size=(1, 1),
                                padding='same', kernel_initializer=init)(act1)
        bn2 = K.layers.BatchNormalization(axis=3)(conv1)
        act2 = K.layers.Activation('relu')(bn2)
        # convolution layer
        conv2 = K.layers.Conv2D(filters=growth_rate, kernel_size=(3, 3),
                                padding='same', kernel_initializer=init)(act2)
        X = K.layers.concatenate([X, conv2])
        nb_filters += growth_rate
    
    return X, nb_filters

In [14]:
if __name__ == '__main__':
    X = K.Input(shape=(56, 56, 64))
    Y, nb_filters = dense_block(X, 64, 32, 6)
    model = K.models.Model(inputs=X, outputs=Y)
    model.summary()
    print(nb_filters)

/home/ediddev/.local/lib/python3.8/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 56, 56, 64)  0           []                               
                                ]                                                                 
                                                                                                  
 batch_normalization_72 (BatchN  (None, 56, 56, 64)  256         ['input_5[0][0]']                
 ormalization)                                                                                    
                                                                                                  
 activation_67 (Activation)     (None, 56, 56, 64)   0           ['batch_normalization_72[0][0]'] 
                                                                                            

In [15]:
"""Task 6: Transition Layer"""

def transition_layer(X, nb_filters, compression):
    """Builds a transition layer as described in Densely Connected
        Convolutional Networks
    Args:
        X: output from the previous layer
        nb_filters: integer representing the number of filters in X
        compression: compression factor for the transition layer
    Returns:
        The output of the transition layer and the number of filters within
            the output, respectively"""

    init = K.initializers.he_normal()
    bn1 = K.layers.BatchNormalization(axis=3)(X)
    act1 = K.layers.Activation('relu')(bn1)
    conv1 = K.layers.Conv2D(filters=int(nb_filters * compression),
                            kernel_size=(1, 1), padding='same',
                            kernel_initializer=init)(act1)
    avg_pool = K.layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2),
                                         padding='valid')(conv1)
    return avg_pool, int(nb_filters * compression)

In [16]:
if __name__ == '__main__':
    X = K.Input(shape=(56, 56, 256))
    Y, nb_filters = transition_layer(X, 256, 0.5)
    model = K.models.Model(inputs=X, outputs=Y)
    model.summary()
    print(nb_filters)

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 56, 56, 256)]     0         
                                                                 
 batch_normalization_84 (Bat  (None, 56, 56, 256)      1024      
 chNormalization)                                                
                                                                 
 activation_79 (Activation)  (None, 56, 56, 256)       0         
                                                                 
 conv2d_84 (Conv2D)          (None, 56, 56, 128)       32896     
                                                                 
 average_pooling2d_1 (Averag  (None, 28, 28, 128)      0         
 ePooling2D)                                                     
                                                                 
Total params: 33,920
Trainable params: 33,408
Non-trainable

In [21]:
"""Task 7: DenseNet-121"""

def densenet121(growth_rate=32, compression=1.0):
    """Builds the DenseNet-121 architecture as described in Densely Connected
        Convolutional Networks
    Args:
        growth_rate: growth rate
        compression: compression factor
    Returns:
        the keras model"""

    init = K.initializers.he_normal()
    inputs = K.Input(shape=(224, 224, 3))
    bn1 = K.layers.BatchNormalization(axis=3)(inputs)
    act1 = K.layers.Activation('relu')(bn1)
    conv1 = K.layers.Conv2D(filters=2 * growth_rate, kernel_size=(7, 7),
                            kernel_initializer=init, padding='same')(act1)
    pool1 = K.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2),
                                  padding='same')(conv1)
    block1, nb_filters = dense_block(pool1, 2 * growth_rate, growth_rate, 6)
    transition_layer1, nb_filters = transition_layer(block1, nb_filters, compression)
    block2, nb_filters = dense_block(transition_layer1, nb_filters, growth_rate, 12)
    transition_layer2, nb_filters = transition_layer(block2, nb_filters, compression)
    block3, nb_filters = dense_block(transition_layer2, nb_filters, growth_rate, 24)
    transition_layer3, nb_filters = transition_layer(block3, nb_filters, compression)
    block4, nb_filters = dense_block(transition_layer3, nb_filters, growth_rate, 16)
    avg_pool = K.layers.AveragePooling2D(pool_size=(7, 7), padding='valid')(block4)
    dense = K.layers.Dense(units=1000, activation='softmax',
                           kernel_initializer=init)(avg_pool)
    model = K.models.Model(inputs=inputs, outputs=dense)
    return model

In [22]:
if __name__ == '__main__':
    model = densenet121(32, 0.5)
    model.summary()

/home/ediddev/.local/lib/python3.8/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 batch_normalization_112 (Batch  (None, 224, 224, 3)  12         ['input_9[0][0]']                
 Normalization)                                                                                   
                                                                                                  
 activation_107 (Activation)    (None, 224, 224, 3)  0           ['batch_normalization_112[0][0]']
                                                                                            